Парсим дынные московский ивентов с сайта 
pro.culture.ru

In [1]:
!pip install pandas numpy lxml bs4 requests

In [2]:
from bs4 import BeautifulSoup
import requests as req
import time
import pandas as pd

In [3]:
def parse_data_heads(html_text):
    theme = html_text.find('div', class_='entity-list-v2_label').text
    title = html_text.find('div', class_='entity-list-v2_title').text
    link = 'https://www.culture.ru{}'.format(html_text.find('a', class_='entity-list-v2_title-link').get('href'))
    text = html_text.find('div', class_='entity-list-v2_description')
    return {'theme': theme, 'title': title, 'link':link, 'text': text}

In [30]:
data = [None,]
result = []
page = 1
while 1:
    link = 'https://www.culture.ru/search?query=%D0%BC%D0%BE%D1%81%D0%BA%D0%B2%D0%B0&page={}&limit=15'.format(page)
    resp = req.get(link)

    soup = BeautifulSoup(resp.text, 'lxml')
    data = soup.find('div', class_='entity-list-v2').findAll('div', class_='js-entity-list-item entity-list-v2_item')
    if len(data) == 0:
        break
    data_parse = [parse_data_heads(i) for i in data]
    result.extend(data_parse)
    page += 1

In [31]:
from time import sleep
res = []
for i in range(len(result)):
    sleep(1)
    data = result[i]
    link = result[i]['link']
    try:
        resp = req.get(link)
    except:
        print("Error_link: {} link: {}".format(i, link))
        continue
    soup = BeautifulSoup(resp.text, 'lxml')
    try:
        if soup.find('div', class_='institute-poster') is None:
            data['image'] = soup.find('div', class_='cover_image-bg').get('style').split('(')[-1][:-1]
        else:
            data['image'] = soup.find(
                'div', class_='institute-poster'
                ).find('img').get('src')
    except:
        print("Error_img: {} link: {}".format(i, link))
        continue
    if soup.find('div', class_='institute-poster_place') is None:
        data['adress'] = None
    else:
        data['adress'] = soup.find('div', class_='institute-poster_place').find('a').text
    res.append(data)


Error_img: 311 link: https://www.culture.ru/institutes/14187/arkhangelskoe-tyurikovo
Error_img: 421 link: https://www.culture.ru/institutes/14255/zakharino-znamenskoe
Error_img: 440 link: https://www.culture.ru/institutes/14144/znamenskoe-sadki
Error_img: 447 link: https://www.culture.ru/institutes/14129/chernyovo
Error_img: 460 link: https://www.culture.ru/institutes/14373/usadba-g-a-zakharina
Error_img: 463 link: https://www.culture.ru/institutes/14344/bolshakovo
Error_img: 709 link: https://www.culture.ru/movies/852/moskva
Error_img: 710 link: https://www.culture.ru/poems/18487/moskva
Error_img: 711 link: https://www.culture.ru/poems/44598/moskva
Error_img: 712 link: https://www.culture.ru/poems/23762/moskva
Error_img: 713 link: https://www.culture.ru/poems/48157/moskva
Error_img: 714 link: https://www.culture.ru/poems/33448/moskva
Error_img: 715 link: https://www.culture.ru/poems/24403/moskva
Error_img: 716 link: https://www.culture.ru/poems/33640/moya-moskva
Error_img: 717 link: h

In [32]:
for i in range(len(res)):
    if res[i]['text'] is not None:
        res[i]['text'] = res[i]['text'].text

In [33]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [34]:
import json
with open('MyDrive/MyDrive/pro_culture.json', 'w') as f:
    json.dump(res, f)